# AI Cyoda configurations Q&A with RAG Langchain

Welcome to this Jupyter notebook! This notebook serves as your guide to developing an AI-powered Question & Answer system using the Langchain library. This system utilizes the Retrieval-Augmented Generation (RAG) model that leverages OpenAI's GPT-3 model to provide intelligent and context-aware responses.

The primary purpose of this notebook is to generate Cyoda related code. It does so by interacting with the data set available in the official Cyoda repository. 

## What will we cover?

In this notebook, we will go through the following steps:

1. **Setting up the environment**: We will install necessary libraries and load environment variables.

2. **Initializing the AI model**: We will initialize the ChatOpenAI model with the appropriate parameters.

3. **Loading instructions and entities**: We will load instructions and entities from the official repository using the GitLoader.

4. **Splitting documents and creating a vectorstore**: We will split the loaded documents into chunks and create a vectorstore using the Chroma library.

5. **Defining prompts for contextualizing and answering questions**: We will define prompts that the AI model will use to contextualize and answer questions.

6. **Creating a retrieval chain**: We will create a retrieval chain that combines the history-aware retriever and the question-answer chain.

7. **Running the chatbot**: Finally, we will run the chatbot and see it in action!

## Let's get started!

Please follow along with the code cells and comments to understand each step of the process. If you have any questions or run into any issues, feel free to ask for help. Happy coding!

Install requirements

In [ ]:
pip install -r ../requirements.txt

### Load environment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
WORK_DIR = os.environ["WORK_DIR"]
API_KEY = os.environ["CYODA_API_KEY"]
API_SECRET = os.environ["CYODA_API_SECRET"]
API_URL = os.environ["CYODA_API_URL"]+"/api"
GRPC_ADDRESS = os.environ["GRPC_ADDRESS"]
WORK_DIR = os.environ["WORK_DIR"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
WORK_DIR = os.environ["WORK_DIR"]
TOKEN = ""

In [ ]:
ENTITY_CLASS_NAME = "com.cyoda.tdb.model.treenode.TreeNodeEntity"
ENTITY_NAME="prizes"
AGG_ENTITY_NAME=ENTITY_NAME+"_agg"
MODEL_VERSION="1001"

In [ ]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
%%script echo skipping
##for google colab (optional)
# This cell is optional and can be skipped
from google.colab import userdata
API_KEY = userdata.get('OPENAI_API_KEY')
WORK_DIR = userdata.get('WORK_DIR')

### Handle unsupported version of sqlite3 (optional)

In [ ]:
pip install pysqlite3-binary

In [ ]:
import sys

__import__("pysqlite3")
sys.modules["sqlite3"] = sys.modules["pysqlite3"]

### Initialize ChatOpenAI

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import GitLoader, DirectoryLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import HumanMessage

In [ ]:
llm = ChatOpenAI(
    temperature=0.85,
    max_tokens=4000,
    model="gpt-4o",
    openai_api_key=OPENAI_API_KEY,
)

### Load instructions and entities from the official cyoda repository

In [ ]:
%%script echo skipping
loader = GitLoader(
    repo_path=WORK_DIR,
    branch="cyoda-ai-configurations-3.0.x",
    file_filter=lambda file_path: file_path.startswith(f"{WORK_DIR}/data/rag/v1/connections/templates"),
)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")

In [ ]:
loader = DirectoryLoader(
    f"{WORK_DIR}/data/v1/connections/templates", loader_cls=TextLoader
)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")


### Split documents and create vectorstore

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
            documents=splits, embedding=OpenAIEmbeddings()
        )
retriever = vectorstore.as_retriever(
            search_kwargs={"k": 10}
        )

In [ ]:
count = vectorstore._collection.count()
print(count)

In [ ]:
res = vectorstore.similarity_search("Get FOREIGN TRAVEL FOR LEEDS CITY COUNCIL")
print(res)

### Define prompts for contextualizing question and answering question

In [ ]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question

In [ ]:
qa_system_prompt = """You are a connection generation assistant. You are provided with connection_dto_template.json with bean com.cyoda.plugins.datasource.dtos.DataSourceConfigsContainerDto template. \
In the template there are blanks <> which you need to fill in after analysing user input. Return the rresulting json without any comments.
{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

### Create retrieval chain

In [ ]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Initialize chat history and relevant functions

In [ ]:
chat_history = {}

In [ ]:
# Function to add a message to the chat history
def add_to_chat_history(id, question, message):
    if id in chat_history:
        chat_history[id].extend([HumanMessage(content=question), message])
    else:
        chat_history[id] = [HumanMessage(content=question), message]

In [ ]:
# Function to clear chat history
def clear_chat_history(id):
    if id in chat_history:
        del chat_history[id]

In [ ]:
def ask_question(id, question):
    ai_msg = rag_chain.invoke(
        {"input": question, "chat_history": chat_history.get(id, [])}
    )
    add_to_chat_history(id, question, ai_msg["answer"])
    return ai_msg["answer"]

### Start a chat session

In [ ]:
import uuid

# Generate a unique ID for the chat session
id = uuid.uuid1()

In [ ]:
##Rewrite CyodaCalculationMemberClient to java. Produce ready to use code. Leave spring boot
question = "Create a connection for GET https://raw.githubusercontent.com/Cyoda-platform/cyoda-ai/cyoda-ai-configurations-3.0.x/config-generation/prizes_entities.json"
result = ask_question(id, question)
print(result)

In [ ]:
print(result)

In [ ]:
print(chat_history)

In [ ]:
# Authenticate with the Cyoda API
import requests
import json

api_url = API_URL + "/auth/login"
headers = {"Content-Type": "application/json", "X-Requested-With": "XMLHttpRequest"}
auth_data = {"username": API_KEY, "password": API_SECRET}
logger.info(api_url)
response = requests.post(api_url, headers=headers, data=json.dumps(auth_data))
if response.status_code == 200:
    logger.info("Authentication successful!")
    TOKEN = response.json().get("token")
else:
    logger.info("Authentication failed. Please check your API credentials.")

In [ ]:
def send_post_request(path, data):
    url = f"{API_URL}/{path}"

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.post(url, headers=headers, data=data)
    return response

In [ ]:
def save_data(data):
    path = f"data-source-config/import-cobi-config?cleanBeforeImport=false&doPostProcess=false"
    response = send_post_request(path=path, data=data)
    logger.info(response)
    return response

In [ ]:
import json
import logging

logger = logging.getLogger(__name__)

def test_save_data(file_path):
    data = ''
    try:
        with open(file_path, 'r') as file:
            data = file.read()
    except Exception as e:
        logger.error(f"Failed to read JSON file: {e}")
        return

    try:
        data = json.loads(data)  # Parse the JSON string into a dictionary
    except json.JSONDecodeError as e:
        logger.error(f"Failed to decode JSON: {e}")
        return

    # Ensure `result` is correctly formatted as a JSON-compatible object
    

    try:
        result_json = json.loads(result)
        data['dataSources'] = result_json['dataSources']
    except json.JSONDecodeError as e:
        logger.error(f"Failed to decode result as JSON: {e}")
        return

    # Convert the modified dictionary back to a JSON string
    data = json.dumps(data, indent=4)
    print(data)

    response = save_data(data=data)
    logger.info(response)
    assert (
        response.status_code == 200
    ), f"Expected 200, got {response.status_code}"

file_path_base = f"{WORK_DIR}/data/v1/connections/templates/connection_dto_template.json"
test_save_data(file_path_base)


In [ ]:
#clear chat history if necessary
clear_chat_history(id)